In [2]:
# ============================================================
# Spillover Interpretation Pipeline (VAR -> GIRF, GFEVD, DY)
# FIXED:
#   - Prevent p=0 (force p>=1)
#   - Use HQIC by default for interpretation stability
#   - Optional: set business-day frequency (off by default)
#
# Outputs (./spillover_outputs):
#   - var_summary.txt
#   - girf_long.csv
#   - GIRF__shock_*.png
#   - gfevd_H{H}.csv
#   - dy_table_H{H}.csv
#   - dy_total_H{H}.txt
#   - run_metadata.csv
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

# -------------------------
# Config
# -------------------------
IN_PATH = "./only_var_input.csv"
OUT_DIR = "./spillover_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

USE_DIFF_COLS_IF_PRESENT = True

MAXLAGS = 8
LAG_SELECT = "hqic"     # "hqic"(recommended) or "aic" or "bic"
FORCE_MIN_LAG_1 = True  # <<< 핵심: p=0 방지

IRF_PERIODS = 30
FEVD_HORIZONS = [7, 15, 30]

# Set to True only if you really want to force a business-day frequency.
# If your series skips holidays/weekends, leaving this False is fine.
FORCE_BUSINESS_DAY_FREQ = False
FILL_METHOD_IF_FREQ_FORCED = "ffill"  # "ffill" or None

# Standardize before estimation? Usually unnecessary for GIRF/GFEVD on dlog series.
STANDARDIZE = False

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# -------------------------
# Helpers
# -------------------------
def choose_var_columns(df, use_diff_like=True):
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    if use_diff_like:
        diff_like = [c for c in numeric_cols if c.startswith("dlog_") or c.startswith("d_")]
        cols = diff_like if len(diff_like) >= 2 else numeric_cols
    else:
        cols = numeric_cols
    cols = sorted(cols)
    if len(cols) < 2:
        raise ValueError(f"Need >=2 numeric columns for VAR. Found: {cols}")
    return cols

def standardize_fit_apply(X):
    mu = X.mean()
    sd = X.std(ddof=0).replace(0, 1.0)
    Xz = (X - mu) / sd
    return Xz, mu, sd

def safe_int(x):
    if x is None:
        return None
    try:
        if isinstance(x, float) and np.isnan(x):
            return None
        return int(x)
    except Exception:
        return None

# -------------------------
# Generalized IRF (Pesaran-Shin)
# GIRF_h(:,j) = Phi_h * Sigma * e_j / sqrt(sigma_jj)
# -------------------------
def compute_girf(var_res, periods=30):
    Sigma = np.asarray(var_res.sigma_u)        # (K,K)
    K = Sigma.shape[0]
    Phi = var_res.ma_rep(periods)              # (periods+1, K, K), includes h=0

    girf = np.zeros((periods + 1, K, K), dtype=float)  # h, response_i, shock_j
    for j in range(K):
        ej = np.zeros((K, 1))
        ej[j, 0] = 1.0
        denom = np.sqrt(Sigma[j, j]) if Sigma[j, j] > 0 else 1.0
        for h in range(periods + 1):
            girf[h, :, j] = (Phi[h] @ Sigma @ ej).flatten() / denom
    return girf

# -------------------------
# Generalized FEVD (Pesaran-Shin / DY-style)
# theta_ij(H) = (1/sigma_jj) * sum_{h=0..H-1} (e_i' Phi_h Sigma e_j)^2
#               / sum_{h=0..H-1} e_i' Phi_h Sigma Phi_h' e_i
# Row-normalize so each i sums to 1 (DY convention).
# -------------------------
def compute_gfevd(var_res, horizon):
    Sigma = np.asarray(var_res.sigma_u)
    K = Sigma.shape[0]
    Phi = var_res.ma_rep(horizon - 1)          # 0..H-1 length H

    num = np.zeros((K, K), dtype=float)
    den = np.zeros(K, dtype=float)

    diag_sigma = np.diag(Sigma)
    diag_sigma = np.where(diag_sigma > 0, diag_sigma, 1.0)

    for h in range(horizon):
        Ph = Phi[h]
        PhSigma = Ph @ Sigma

        den += np.diag(Ph @ Sigma @ Ph.T)

        # (e_i' Ph Sigma e_j)^2 / sigma_jj
        num += (PhSigma ** 2) / diag_sigma[None, :]

    den = np.where(den > 0, den, 1e-12)
    theta = num / den[:, None]

    row_sums = theta.sum(axis=1, keepdims=True)
    row_sums = np.where(row_sums > 0, row_sums, 1e-12)
    theta_norm = theta / row_sums
    return theta_norm

def dy_spillover_table(theta_norm, var_names):
    K = len(var_names)
    off_diag_sum = theta_norm.sum() - np.trace(theta_norm)
    total = 100.0 * off_diag_sum / K

    to_others = 100.0 * (theta_norm.sum(axis=1) - np.diag(theta_norm)) / K
    from_others = 100.0 * (theta_norm.sum(axis=0) - np.diag(theta_norm)) / K
    net = to_others - from_others

    tbl = pd.DataFrame({
        "variable": var_names,
        "to_others(%)": to_others,
        "from_others(%)": from_others,
        "net(%)": net
    })
    return total, tbl

def plot_girf(girf, var_names, out_dir):
    Hp1, K, _ = girf.shape
    horizons = np.arange(Hp1)

    for shock_j, shock_name in enumerate(var_names):
        plt.figure(figsize=(10, 6))
        for resp_i, resp_name in enumerate(var_names):
            plt.plot(horizons, girf[:, resp_i, shock_j], label=resp_name)
        plt.axhline(0.0, linewidth=1)
        plt.title(f"GIRF responses to 1-s.d. shock in: {shock_name}")
        plt.xlabel("Horizon (days)")
        plt.ylabel("Response")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, f"GIRF__shock_{shock_name}.png"), dpi=160)
        plt.close()

# -------------------------
# Main
# -------------------------
df = pd.read_csv(IN_PATH)

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date").reset_index(drop=True)
    df = df.set_index("Date")

df = df.dropna().copy()

var_cols = choose_var_columns(df, use_diff_like=USE_DIFF_COLS_IF_PRESENT)
X = df[var_cols].copy()

# Optional: enforce frequency
if FORCE_BUSINESS_DAY_FREQ and isinstance(X.index, pd.DatetimeIndex):
    X = X.asfreq("B")
    if FILL_METHOD_IF_FREQ_FORCED == "ffill":
        X = X.ffill()
    X = X.dropna()

print("[INFO] Using columns:", var_cols)
if isinstance(X.index, pd.DatetimeIndex):
    print("[INFO] Date range:", X.index.min(), "~", X.index.max())
print("[INFO] n =", len(X))

# Optional standardization
if STANDARDIZE:
    Xz, mu, sd = standardize_fit_apply(X)
else:
    Xz = X
    mu = sd = None

# Fit VAR
model = VAR(Xz)

order = model.select_order(MAXLAGS)
selected = safe_int(getattr(order, LAG_SELECT, None))

# Fallback chain
if selected is None:
    for crit in ["hqic", "bic", "aic"]:
        selected = safe_int(getattr(order, crit, None))
        if selected is not None:
            LAG_SELECT = crit
            break

if selected is None:
    raise RuntimeError("Lag selection failed. Try reducing MAXLAGS or checking data.")

p = int(selected)
if FORCE_MIN_LAG_1:
    p = max(1, p)

print(f"[INFO] Selected lag p={p} by {LAG_SELECT.upper()} (maxlags={MAXLAGS}, forced p>=1={FORCE_MIN_LAG_1})")

res = model.fit(p)

# Save summary
with open(os.path.join(OUT_DIR, "var_summary.txt"), "w", encoding="utf-8") as f:
    f.write(str(res.summary()))

# -------------------------
# 1) GIRF
# -------------------------
girf = compute_girf(res, periods=IRF_PERIODS)  # (H+1, K, K)

# Save GIRF long
K = len(var_cols)
rows = []
for h in range(IRF_PERIODS + 1):
    for i in range(K):
        for j in range(K):
            rows.append({
                "horizon": h,
                "response": var_cols[i],
                "shock": var_cols[j],
                "girf": float(girf[h, i, j])
            })
girf_df = pd.DataFrame(rows)
girf_df.to_csv(os.path.join(OUT_DIR, "girf_long.csv"), index=False)

# Plot GIRF
plot_girf(girf, var_cols, OUT_DIR)
print(f"[INFO] GIRF saved: girf_long.csv and GIRF__shock_*.png in {OUT_DIR}")

# -------------------------
# 2) GFEVD + DY
# -------------------------
for H in FEVD_HORIZONS:
    theta = compute_gfevd(res, horizon=H)
    fevd = pd.DataFrame(theta, index=var_cols, columns=var_cols)
    fevd.to_csv(os.path.join(OUT_DIR, f"gfevd_H{H}.csv"))

    total, dy_tbl = dy_spillover_table(theta, var_cols)
    dy_tbl.to_csv(os.path.join(OUT_DIR, f"dy_table_H{H}.csv"), index=False)

    with open(os.path.join(OUT_DIR, f"dy_total_H{H}.txt"), "w", encoding="utf-8") as f:
        f.write(f"Total spillover index (DY, generalized FEVD), H={H}: {total:.6f}\n")

    print(f"[INFO] H={H} GFEVD saved: gfevd_H{H}.csv")
    print(f"[INFO] H={H} DY saved: dy_table_H{H}.csv, dy_total_H{H}.txt")

# -------------------------
# Metadata
# -------------------------
meta = {
    "IN_PATH": IN_PATH,
    "OUT_DIR": OUT_DIR,
    "COLUMNS_USED": var_cols,
    "N": int(len(X)),
    "DATE_MIN": str(X.index.min()) if isinstance(X.index, pd.DatetimeIndex) else "",
    "DATE_MAX": str(X.index.max()) if isinstance(X.index, pd.DatetimeIndex) else "",
    "MAXLAGS": MAXLAGS,
    "LAG_SELECT": LAG_SELECT,
    "SELECTED_P": int(p),
    "FORCE_MIN_LAG_1": FORCE_MIN_LAG_1,
    "IRF_PERIODS": IRF_PERIODS,
    "FEVD_HORIZONS": FEVD_HORIZONS,
    "STANDARDIZE": STANDARDIZE,
    "FORCE_BUSINESS_DAY_FREQ": FORCE_BUSINESS_DAY_FREQ,
    "RANDOM_STATE": RANDOM_STATE,
}
pd.Series(meta).to_csv(os.path.join(OUT_DIR, "run_metadata.csv"), header=False)

print(f"[DONE] Spillover outputs are in: {OUT_DIR}")


[INFO] Using columns: ['d_UST10Y', 'dlog_COPPER', 'dlog_DXY', 'dlog_SOLVPN', 'dlog_VIX']
[INFO] Date range: 2020-10-13 00:00:00 ~ 2026-01-12 00:00:00
[INFO] n = 1325
[INFO] Selected lag p=1 by HQIC (maxlags=8, forced p>=1=True)


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[INFO] GIRF saved: girf_long.csv and GIRF__shock_*.png in ./spillover_outputs
[INFO] H=7 GFEVD saved: gfevd_H7.csv
[INFO] H=7 DY saved: dy_table_H7.csv, dy_total_H7.txt
[INFO] H=15 GFEVD saved: gfevd_H15.csv
[INFO] H=15 DY saved: dy_table_H15.csv, dy_total_H15.txt
[INFO] H=30 GFEVD saved: gfevd_H30.csv
[INFO] H=30 DY saved: dy_table_H30.csv, dy_total_H30.txt
[DONE] Spillover outputs are in: ./spillover_outputs
